In [17]:
import mkl_data_processing as mkldp
import os
import pickle
import pandas as pd
import sys
import sys
from memory_profiler import profile
import pandas as pd
import numpy as np

sys.path.append('/home/ak/Documents/Research/PaperCode/singlekernelclf/')
import jsonpickle
from MKLpy.preprocessing import normalization, rescale_01
from MKLpy.metrics import pairwise
from MKLpy import generators
from MKLpy.algorithms import AverageMKL, EasyMKL
from sklearn.svm import SVC
from collections import OrderedDict
import os
from sklearn.metrics import accuracy_score
import torch
from MKLpy.model_selection import cross_val_score
from MKLpy.multiclass import OneVsRestMKLClassifier, OneVsOneMKLClassifier

from MKLpy.algorithms import AverageMKL, EasyMKL, \
    KOMD  # KOMD is not a MKL algorithm but a simple kernel machine like the SVM

from fileutils import new_feature_utils as nfu
from fileutils.new_feature_utils import CreateMarketFeatures
import multiprocessing



In [18]:
from fileutils import DataLoader as DataLoader
from fileutils import paths
from fileutils import new_feature_utils as nfu
from fileutils.new_feature_utils import CreateMarketFeatures

In [19]:
def open_pickle_filepath(pickle_file):
    pickle_to_file = pickle.load(open(pickle_file, "rb"), encoding='latin1')

    return pickle_to_file


def forward_Dates(list_of_keys, current_date):
    """
    return all the forward looking dates for each idxKey we use for training

    :param list_of_keys: dates i have model dates for out of sample
    :param current_date: current model date
    :return: forward dates for applying the fitted model
    """
    lookAheadKeys = sorted(i for i in list_of_keys if i > current_date)
    return lookAheadKeys


In [20]:
mainPath = mkldp.paths('main')

symbols = sorted(os.listdir(mkldp.paths('symbols_features')))
jointFeatureLocation = os.path.join(mainPath, "ExperimentCommonLocs/JointLocationsDicts")
commonLocs  = os.path.join(mainPath, "ExperimentCommonLocs/")

mklOOSPredictionPath = os.path.join(mainPath, "ExperimentCommonLocs/MKLOOSPredictions")
allFiles = os.listdir(jointFeatureLocation)

In [21]:
labelsAlternateLocs = {f:os.path.join(commonLocs, f) for f in os.listdir(os.path.join(mainPath, "ExperimentCommonLocs/")) if str("LabelsAlternate") in f }
labelsAlternateLocs

{'LabelsAlternateThree': '/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateThree',
 'LabelsAlternateFour': '/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateFour',
 'LabelsAlternateSeven': '/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateSeven',
 'LabelsAlternateOne': '/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateOne',
 'LabelsAlternateSix': '/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateSix',
 'LabelsAlternateTwo': '/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateTwo',
 'LabelsAlternateFive': '/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateFive'}

In [22]:

alternate_labels_nos = {1:"LabelsAlternateOne", 2:"LabelsAlternateTwo", 3:"LabelsAlternateThree", 
                        4:"LabelsAlternateFour", 5:"LabelsAlternateFive", 6:"LabelsAlternateSix", 
                        7:"LabelsAlternateSeven"}  # we have 7 alternative data types

fittedModelsPath = os.path.join(mainPath, "ExperimentCommonLocs/FittedModels")
oosPredictionsPath = os.path.join(mainPath, "ExperimentCommonLocs/OOSPredictions")

In [62]:
label_idx = 1  # to be serialised

labelsLocation = labelsAlternateLocs[alternate_labels_nos[label_idx]]

labelsFiles = sorted(os.listdir(labelsLocation))

In [67]:
symbol_idx = 1  # pick a symbol
symbol = sorted(symbols)[symbol_idx]  # to be serialised so read all the symbols
print(symbol)

symbolData = DataLoader(mainPath, symbol)  # initiate a path where all the data should be
pickled_models = [f for f in os.listdir(fittedModelsPath) if str(symbol) in f]  # list of all the pickled models



APF.L


In [73]:
print(labelsLocation)
files = os.listdir(jointFeatureLocation)
fileIdx =2
pickle_path = os.path.join( jointFeatureLocation, [f for f in os.listdir(jointFeatureLocation) if str(symbol) in f][fileIdx])

/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateOne


In [74]:
print(pickle_path)

/media/ak/DataOnly/ExperimentCommonLocs/JointLocationsDicts/APF.L_hmm_date:_20170731_feature_date_20170929_label_id:_2_ProcessedData.pkl


In [70]:
print(pickle_path)

/media/ak/DataOnly/ExperimentCommonLocs/JointLocationsDicts/APF.L_hmm_date:_20170731_feature_date_20170929_label_id:_2_ProcessedData.pkl


In [71]:
Datekeys =list(open_pickle_filepath(pickle_path).keys())
print(len(Datekeys))

33


In [75]:
file = files[4]
select_file_path = os.path.join(jointFeatureLocation, file)  # formulate the path
print('Symbol:----->', file.split("_")[0])
symbol = file.split("_")[0]

Symbol:-----> RR.L


In [76]:
select_hmm_date = select_file_path.split("_")[3]  # pull out the hmm_date - strip it out

select_feature_label_date = select_file_path.split("_")[6]  # pull out the label_feature_date

select_label_idx = select_file_path.split("_")[9]  # pull out the label _idx

unpickled_select_file = open_pickle_filepath(select_file_path)  # unplickle the select file

hmm_keys = sorted(list(unpickled_select_file.keys()))  # hmm keys for the select file.

In [78]:
for hmm_date_key in hmm_keys:  # pick and hmm date
    feature_label_keys = sorted(
        unpickled_select_file[hmm_date_key].keys())  # each key here unlocks a feature and label set
    


In [84]:
for hmm_date_key in hmm_keys:  # pick and hmm date
    print('###################', hmm_date_key, '########################')
    feature_label_keys = sorted(
        unpickled_select_file[hmm_date_key].keys())  # each key here unlocks a feature and label set    
    print(feature_label_keys)
    print('#######################################')

    for feature_label_date in feature_label_keys:  # make a list of all the feature dates
        features_file_path = unpickled_select_file[hmm_date_key][feature_label_date][
            0]  # this is the feature path
        labels_file_path = unpickled_select_file[hmm_date_key][feature_label_date][1]  # this is the labels path

        if os.path.isfile(features_file_path):  # if label file exists I can traing
            print('ok----->', feature_label_date)  # if you got to this point we have data so we can mov eon
            labels = pd.read_csv(labels_file_path)  # open labels file
            label_name = str(labels.columns[labels.columns.str.contains(pat='label')].values[0])
            features = open_pickle_filepath(features_file_path)  # opens features file
            hmm_features = nfu.hmm_features_df(features)  # get the hmm features out, so unpack the tuples!
            print('loaded features and labels ')
        else:
            print('PROBLEM----->in one of of your locations')
            continue

################### 20170703 ########################
['20170704', '20170705', '20170706', '20170707', '20170710', '20170711', '20170712', '20170713', '20170714', '20170717', '20170718', '20170719', '20170720', '20170721', '20170724', '20170725', '20170726', '20170727', '20170728', '20170731', '20170801', '20170802', '20170803', '20170804', '20170807', '20170808', '20170809', '20170810', '20170811', '20170814', '20170815', '20170816', '20170817', '20170818', '20170821', '20170822', '20170823', '20170824', '20170825', '20170829', '20170830', '20170831', '20170901', '20170904', '20170905', '20170906', '20170907', '20170908', '20170911', '20170912', '20170913', '20170914', '20170915', '20170918', '20170919', '20170920', '20170921', '20170922', '20170925', '20170926', '20170927', '20170928', '20170929', '20180301', '20180302', '20180305', '20180306', '20180307', '20180308', '20180309', '20180312', '20180313', '20180314', '20180315', '20180316', '20180319', '20180320', '20180321', '20180322

FileNotFoundError: [Errno 2] File /media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateSeven/RR.L/20170704.csv does not exist: '/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateSeven/RR.L/20170704.csv'

In [86]:
os.listdir('/media/ak/DataOnly/ExperimentCommonLocs/LabelsAlternateSeven/')

[]